In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
small_data = pd.read_csv('small_data.csv')
small_data

,Group,Dimethylglycine_HMDB0000092,Glyoxylic acid_HMDB0000119,Pyruvic acid_HMDB0000243,L-Lactic acid_HMDB0000190,Alpha-Hydroxyisobutyric acid_HMDB0000729,Glycolic acid_HMDB0000115,Caproic acid_HMDB0000535,L-Alanine_HMDB0000161,Alpha-ketoisovaleric acid_HMDB0000019,...,Sucrose_HMDB0000258,MG(18:0e/0:0/0:0)_HMDB0011143,Trehalose_HMDB0000975,D-Maltose_HMDB0000163,Lactitol_HMDB0040937,Xanthylic acid_HMDB0001554,Adenosine monophosphate_HMDB0000045,Cholesterol_HMDB0000067,Lathosterol_HMDB0001170,7b-Hydroxycholesterol_ HMDB0006119
0,0,1426172,1591380,100740,1972502,80250,3891814,2386481,513996,18467,...,1582940,226467,2250788,856000,38063,364001,7301,5644816,23558,525597
1,0,1517956,1686277,142948,2729996,85646,6069966,2952691,517222,23619,...,1453751,279571,439349,205406,39952,134353,14407,6020618,23632,535842
2,0,1664485,1827098,130359,2211164,94101,5831749,2782558,567592,17357,...,1360706,251630,366872,158065,51457,195269,7742,6263993,24783,554371
3,0,1159526,1294607,93469,1557122,63105,4042984,1781499,448708,15182,...,699933,191838,224136,76907,39453,407070,5052,5277272,18253,480930
4,1,676603,762540,174855,3666034,96213,3692688,1314990,774759,21773,...,2377553,239961,2110805,831730,50190,9796,1551,5116960,20178,428768
5,1,727034,807206,126424,2724566,66314,3069220,709370,698348,19513,...,1849690,158963,512520,216781,46054,129625,1199,5653514,21400,316166
6,1,751274,832044,150530,4151215,87792,3886825,1073135,856665,22056,...,1640040,230621,572257,232393,51065,50165,3331,6235203,25484,443281
7,1,593974,644762,149130,4112542,94976,4503385,658502,791352,25182,...,1702734,261009,611152,255378,53199,6913,1454,6238185,28553,435282
8,2,560734,625327,128295,2821701,88315,3315107,655856,579283,15528,...,2555027,292232,405252,170659,43887,4526,113,6015531,34844,10153
9,2,645725,690614,165197,2506349,90194,5603085,976099,546050,16290,...,2743023,281899,478809,209904,53412,24346,2641,7386359,37677,13283


# Whether storage condition matter?

In [3]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

X = small_data.iloc[:,2:]
scaler = MinMaxScaler()
scaler.fit(X)
X_centered = scaler.transform(X)

y = small_data['Group']
y = pd.factorize(y)[0]
pca = PCA(n_components=5)
pca.fit(X_centered)

X_PCA = pca.transform(X)
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

[0.42024247 0.21040516 0.10529459 0.05272396 0.04362888]
0.8322950609557146
[10.19369239  7.2128972   5.10251846  3.61065128  3.28449498]


In [25]:
X = small_data.iloc[:,2:]
scaler = MinMaxScaler()
scaler.fit(X)
X_centered = scaler.transform(X)

y = small_data['Group']
y = pd.factorize(y)[0]
pca = PCA(n_components=5)
pca.fit(X)
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))

[0.66937651 0.10797663 0.06695484 0.04443856 0.036998  ]
0.9257445520262454


### explained variance by the first 5 PCs decreases if I centered the columns... why?

In [61]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
loo.get_n_splits(X_centered)

print(loo)
y_pred = []
loo_decision_func = []
for train_index, test_index in loo.split(X_centered):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_centered[train_index], X_centered[test_index]
    y_train, y_test = y[train_index], y[test_index]

    lsvc = LinearSVC(C = 0.05)
    lsvc.fit(X_train, y_train)
    print('true:',y_test)
    pred = lsvc.predict(X_test)
    y_pred.append(pred[0])
    print('predicted: ',pred)
    dec_func = lsvc.decision_function(X_test)
    print('decision function:',dec_func)
    loo_decision_func.append(dec_func[0])


LeaveOneOut()
TRAIN: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14] TEST: [0]
true: [0]
predicted:  [0]
decision function: [[-0.15019752 -0.68407323 -0.95223052 -0.22322374]]
TRAIN: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14] TEST: [1]
true: [0]
predicted:  [3]
decision function: [[-0.27984141 -1.07291517 -1.14675926  0.08698574]]
TRAIN: [ 0  1  3  4  5  6  7  8  9 10 11 12 13 14] TEST: [2]
true: [0]
predicted:  [0]
decision function: [[-0.08141998 -1.01553965 -0.95373016 -0.22504922]]
TRAIN: [ 0  1  2  4  5  6  7  8  9 10 11 12 13 14] TEST: [3]
true: [0]
predicted:  [3]
decision function: [[ 0.02840833 -0.60586815 -0.79836574  0.09781234]]
TRAIN: [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14] TEST: [4]
true: [1]
predicted:  [1]
decision function: [[-0.75861084  0.39645883 -0.71035026 -1.08778119]]
TRAIN: [ 0  1  2  3  4  6  7  8  9 10 11 12 13 14] TEST: [5]
true: [1]
predicted:  [1]
decision function: [[-0.68260951  0.5109546  -0.41681156 -0.96388168]]
TRAIN: [ 0  1  2  3  4  5  7  8  9 10 

In [68]:
loo_decision_func_df = pd.DataFrame(loo_decision_func)
loo_decision_func_df.to_csv('loo_decicion_function.csv')

In [52]:
print(y_pred)
print(y)

[0, 3, 0, 3, 1, 1, 1, 1, 2, 2, 2, 0, 0, 0, 0]
[0 0 0 0 1 1 1 1 2 2 2 3 3 3 3]


In [57]:
lsvc = LinearSVC(C = 0.05)
lsvc.fit(X_centered, y)
lsvc.decision_function(X_centered)

array([[ 0.35023824, -0.82537669, -0.97321099, -0.56514329],
       [ 0.39546938, -1.07293199, -1.14675059, -0.49596696],
       [ 0.34996235, -1.01557289, -0.97389152, -0.52873964],
       [ 0.38881872, -0.75713872, -0.87211414, -0.30703989],
       [-0.87776853,  0.68660718, -0.8491922 , -1.08779343],
       [-0.82782463,  0.73894019, -0.68886019, -0.98768765],
       [-0.95673418,  0.80231775, -0.90784458, -1.28094184],
       [-1.0290455 ,  0.68394527, -0.77587512, -1.11590289],
       [-0.93925979, -0.54730186,  0.64810768, -0.80048331],
       [-1.10313275, -0.81252337,  0.62773718, -0.89156572],
       [-0.83017705, -0.6814257 ,  0.55235533, -1.02364986],
       [-0.53002766, -0.84911148, -0.91481018,  0.46409772],
       [-0.62915081, -0.86722946, -0.83578801,  0.51990111],
       [-0.39798548, -0.9948801 , -0.98400036,  0.27917171],
       [-0.14533667, -1.15392955, -0.85881395,  0.29056065]])

In [56]:
decision_func = pd.DataFrame(lsvc.decision_function(X_centered))
decision_func.to_csv('decision_function_LinearSVC.csv')

# Feature analysis

In [61]:
small_data = pd.read_csv('small_data.csv')
small_data

,Group,Dimethylglycine_HMDB0000092,Glyoxylic acid_HMDB0000119,Pyruvic acid_HMDB0000243,L-Lactic acid_HMDB0000190,Alpha-Hydroxyisobutyric acid_HMDB0000729,Glycolic acid_HMDB0000115,Caproic acid_HMDB0000535,L-Alanine_HMDB0000161,Alpha-ketoisovaleric acid_HMDB0000019,...,Sucrose_HMDB0000258,MG(18:0e/0:0/0:0)_HMDB0011143,Trehalose_HMDB0000975,D-Maltose_HMDB0000163,Lactitol_HMDB0040937,Xanthylic acid_HMDB0001554,Adenosine monophosphate_HMDB0000045,Cholesterol_HMDB0000067,Lathosterol_HMDB0001170,7b-Hydroxycholesterol_ HMDB0006119
0,0,1426172,1591380,100740,1972502,80250,3891814,2386481,513996,18467,...,1582940,226467,2250788,856000,38063,364001,7301,5644816,23558,525597
1,0,1517956,1686277,142948,2729996,85646,6069966,2952691,517222,23619,...,1453751,279571,439349,205406,39952,134353,14407,6020618,23632,535842
2,0,1664485,1827098,130359,2211164,94101,5831749,2782558,567592,17357,...,1360706,251630,366872,158065,51457,195269,7742,6263993,24783,554371
3,0,1159526,1294607,93469,1557122,63105,4042984,1781499,448708,15182,...,699933,191838,224136,76907,39453,407070,5052,5277272,18253,480930
4,1,676603,762540,174855,3666034,96213,3692688,1314990,774759,21773,...,2377553,239961,2110805,831730,50190,9796,1551,5116960,20178,428768
5,1,727034,807206,126424,2724566,66314,3069220,709370,698348,19513,...,1849690,158963,512520,216781,46054,129625,1199,5653514,21400,316166
6,1,751274,832044,150530,4151215,87792,3886825,1073135,856665,22056,...,1640040,230621,572257,232393,51065,50165,3331,6235203,25484,443281
7,1,593974,644762,149130,4112542,94976,4503385,658502,791352,25182,...,1702734,261009,611152,255378,53199,6913,1454,6238185,28553,435282
8,2,560734,625327,128295,2821701,88315,3315107,655856,579283,15528,...,2555027,292232,405252,170659,43887,4526,113,6015531,34844,10153
9,2,645725,690614,165197,2506349,90194,5603085,976099,546050,16290,...,2743023,281899,478809,209904,53412,24346,2641,7386359,37677,13283


In [62]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

small_data = pd.read_csv('small_data.csv')
X = small_data.iloc[:,1:]
scaler = MinMaxScaler()
scaler.fit(X)
X_centered = scaler.transform(X)

y = small_data['Group']
y = pd.factorize(y)[0]

Xctd_df = pd.DataFrame(X_centered)

In [38]:
# Xctd_df.iloc[14,184]
import numpy as np
from sklearn.linear_model import LinearRegression
score_lst = []
for c in range(185):
    reg = LinearRegression().fit(y.reshape(-1, 1), Xctd_df.iloc[:,c])
    score_lst.append(reg.score(y.reshape(-1, 1), Xctd_df.iloc[:,c]))


In [64]:
small_data.columns[1:]

Index(['Dimethylglycine_HMDB0000092', 'Glyoxylic acid_HMDB0000119',
       'Pyruvic acid_HMDB0000243', 'L-Lactic acid_HMDB0000190',
       'Alpha-Hydroxyisobutyric acid_HMDB0000729', 'Glycolic acid_HMDB0000115',
       'Caproic acid_HMDB0000535', 'L-Alanine_HMDB0000161',
       'Alpha-ketoisovaleric acid_HMDB0000019', 'Glycine_HMDB0000123',
       ...
       'Sucrose_HMDB0000258', 'MG(18:0e/0:0/0:0)_HMDB0011143',
       'Trehalose_HMDB0000975', 'D-Maltose_HMDB0000163',
       'Lactitol_HMDB0040937', 'Xanthylic acid_HMDB0001554',
       'Adenosine monophosphate_HMDB0000045', 'Cholesterol_HMDB0000067',
       'Lathosterol_HMDB0001170', '7b-Hydroxycholesterol_ HMDB0006119'],
      dtype='object', length=186)

In [50]:
sorted_lst = sorted(score_lst,reverse=True)
met_index = []
met_name_lst = []
for r in sorted_lst:
    indx = score_lst.index(r)
    met_index.append(indx)
    met_name_lst.append(small_data.columns[1:][indx])



In [52]:
metabolite_r2 = pd.DataFrame({'metabolite name':met_name_lst, 'R_square':sorted_lst})

In [53]:
metabolite_r2.to_excel('metabolite_r2.xlsx')

In [61]:
small_data.iloc[11:,:]

,Group,Dimethylglycine_HMDB0000092,Glyoxylic acid_HMDB0000119,Pyruvic acid_HMDB0000243,L-Lactic acid_HMDB0000190,Alpha-Hydroxyisobutyric acid_HMDB0000729,Glycolic acid_HMDB0000115,Caproic acid_HMDB0000535,L-Alanine_HMDB0000161,Alpha-ketoisovaleric acid_HMDB0000019,...,Sucrose_HMDB0000258,MG(18:0e/0:0/0:0)_HMDB0011143,Trehalose_HMDB0000975,D-Maltose_HMDB0000163,Lactitol_HMDB0040937,Xanthylic acid_HMDB0001554,Adenosine monophosphate_HMDB0000045,Cholesterol_HMDB0000067,Lathosterol_HMDB0001170,7b-Hydroxycholesterol_ HMDB0006119
11,3,1080539,1201990,117525,1409613,71354,3202735,1332556,499903,15218,...,1993969,222420,8848515,3354293,53638,360786,8572,5729825,24820,390464
12,3,1399573,1557938,98968,1282860,66754,3560592,1625191,421395,17876,...,1860120,193325,786710,310727,44281,446259,4900,6538775,26001,410805
13,3,2075023,2308202,118428,1765898,92520,5017534,2137935,527216,21984,...,966079,265925,604507,242247,89800,128876,12029,6996995,29921,478722
14,3,1376262,1518899,133935,1568057,80930,5647582,2417288,433660,18470,...,962019,257435,304054,136614,42446,139153,10073,5841090,26089,413517


# Can one-way ANOVA answer the question?

In [15]:
from scipy.stats import f_oneway
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

small_data = pd.read_csv('small_data.csv')
X = small_data.iloc[:,1:]
scaler = MinMaxScaler()
scaler.fit(X)
X_centered = scaler.transform(X)

y = small_data['Group']
y = pd.factorize(y)[0]

Xctd_df = pd.DataFrame(X_centered)
met_lst = []
f_lst = []
p_lst = []
for c in range(186):
    g0 = Xctd_df.iloc[0:4,c]
    g1 = Xctd_df.iloc[4:8,c]
    g2 = Xctd_df.iloc[8:11,c]
    g3 = Xctd_df.iloc[11:,c]
    f,p = f_oneway(g0, g1, g2, g3)
    met_lst.append(X.columns[c])
    f_lst.append(f)
    p_lst.append(p)


In [18]:
metabolite_f = pd.DataFrame({'metabolite name':met_lst, 'F_score':f_lst,'P_value':p_lst}).sort_values(by='P_value',ascending=True)
metabolite_f.to_excel('metabolite_f.xlsx')

In [19]:
metabolite_f

,metabolite name,F_score,P_value
169,Arachidonic acid_HMDB0001043,223.569290,3.894500e-10
84,2-Aminobenzoic acid_HMDB0001123,190.517510,9.233940e-10
185,7b-Hydroxycholesterol_ HMDB0006119,100.416068,2.830777e-08
80,Phosphoenolpyruvic acid_HMDB0000263,97.183556,3.365516e-08
83,Triethanolamine_HMDB0032538,94.727047,3.852847e-08
...,...,...,...
104,Ribitol_HMDB0000508,0.177038,9.097241e-01
12,2-Hydroxybutyric acid_HMDB0000008,0.155017,9.242675e-01
51,Mesaconic acid_HMDB0000749,0.135694,9.366693e-01
175,D-Sedoheptulose 7-phosphate_HMDB0001068,0.074207,9.726006e-01


# Use SVM approach to find whether a metabolite is affected by storage condition

In [20]:
import pandas as pd
import numpy as np
import sklearn

In [21]:
small_data = pd.read_csv('small_data.csv')
small_data

,Group,Dimethylglycine_HMDB0000092,Glyoxylic acid_HMDB0000119,Pyruvic acid_HMDB0000243,L-Lactic acid_HMDB0000190,Alpha-Hydroxyisobutyric acid_HMDB0000729,Glycolic acid_HMDB0000115,Caproic acid_HMDB0000535,L-Alanine_HMDB0000161,Alpha-ketoisovaleric acid_HMDB0000019,...,Sucrose_HMDB0000258,MG(18:0e/0:0/0:0)_HMDB0011143,Trehalose_HMDB0000975,D-Maltose_HMDB0000163,Lactitol_HMDB0040937,Xanthylic acid_HMDB0001554,Adenosine monophosphate_HMDB0000045,Cholesterol_HMDB0000067,Lathosterol_HMDB0001170,7b-Hydroxycholesterol_ HMDB0006119
0,0,1426172,1591380,100740,1972502,80250,3891814,2386481,513996,18467,...,1582940,226467,2250788,856000,38063,364001,7301,5644816,23558,525597
1,0,1517956,1686277,142948,2729996,85646,6069966,2952691,517222,23619,...,1453751,279571,439349,205406,39952,134353,14407,6020618,23632,535842
2,0,1664485,1827098,130359,2211164,94101,5831749,2782558,567592,17357,...,1360706,251630,366872,158065,51457,195269,7742,6263993,24783,554371
3,0,1159526,1294607,93469,1557122,63105,4042984,1781499,448708,15182,...,699933,191838,224136,76907,39453,407070,5052,5277272,18253,480930
4,1,676603,762540,174855,3666034,96213,3692688,1314990,774759,21773,...,2377553,239961,2110805,831730,50190,9796,1551,5116960,20178,428768
5,1,727034,807206,126424,2724566,66314,3069220,709370,698348,19513,...,1849690,158963,512520,216781,46054,129625,1199,5653514,21400,316166
6,1,751274,832044,150530,4151215,87792,3886825,1073135,856665,22056,...,1640040,230621,572257,232393,51065,50165,3331,6235203,25484,443281
7,1,593974,644762,149130,4112542,94976,4503385,658502,791352,25182,...,1702734,261009,611152,255378,53199,6913,1454,6238185,28553,435282
8,2,560734,625327,128295,2821701,88315,3315107,655856,579283,15528,...,2555027,292232,405252,170659,43887,4526,113,6015531,34844,10153
9,2,645725,690614,165197,2506349,90194,5603085,976099,546050,16290,...,2743023,281899,478809,209904,53412,24346,2641,7386359,37677,13283


In [34]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

X = small_data.iloc[:,1:]
scaler = MinMaxScaler()
scaler.fit(X)
X_centered = scaler.transform(X)

y = small_data['Group']
y = pd.factorize(y)[0]

In [37]:
len(X_centered)

15

In [39]:
lsvc = LinearSVC(C = 0.05,class_weight='balanced')
lsvc.fit(X_centered, y)

lsvc.score(X_centered,y)


1.0

In [59]:
lsvc.coef_[2]

array([-0.03231507, -0.03338123, -0.00676694, -0.00997502,  0.01663275,
        0.02306413, -0.03529797, -0.02268414, -0.05565251, -0.01801556,
        0.00348403, -0.00202924, -0.00283253,  0.01459012,  0.02757421,
       -0.00807228,  0.01193908,  0.00220507,  0.01819767, -0.00177881,
       -0.01433871,  0.01691421, -0.02738069, -0.02042852, -0.01457856,
        0.02038369,  0.01624843, -0.03560869, -0.00277936,  0.01157576,
       -0.02737014,  0.04354427, -0.01232663,  0.00549356, -0.02420722,
        0.00457225,  0.01130667, -0.007241  , -0.01314273, -0.00556695,
       -0.01351231, -0.0195025 , -0.00387176, -0.00672387,  0.04411719,
        0.00074375, -0.00835907, -0.03635017, -0.02217886, -0.02261273,
       -0.02620515, -0.01101129, -0.06889758, -0.02923355,  0.01089049,
        0.04835253,  0.0529601 , -0.0264839 , -0.02339349, -0.01239978,
       -0.03827279,  0.00065957, -0.02400988,  0.00203953, -0.04112139,
       -0.02170709, -0.00116194, -0.02008072, -0.01953858, -0.03

In [71]:
aha = pd.DataFrame({'metabolite':X.columns,'coef0':lsvc.coef_[0],'coef1':lsvc.coef_[1],'coef2':lsvc.coef_[2],'coef3':lsvc.coef_[3]})
# aha['avg_coef'] = abs(aha['coef0'] + aha['coef1'] + aha['coef2'] + aha['coef3'])/4
aha['avg_coef'] = (abs(aha['coef0']) + abs(aha['coef1']) + abs(aha['coef2']) + abs(aha['coef3']))/4
aha = aha.sort_values(by='avg_coef',ascending=False)
aha.to_excel('SVM_coef.xlsx')
# aha.sort_values(by='avg_coef',ascending=False)

In [70]:
aha.sort_values(by='avg_coef',ascending=False)

,metabolite,coef0,coef1,coef2,coef3,avg_coef
158,Heptadecanoic acid_HMDB0002259,0.120888,0.013514,-0.044474,-0.093227,0.068026
31,Benzoic acid_HMDB0001870,0.084479,-0.047435,0.043544,-0.075546,0.062751
114,O-Phosphoethanolamine_HMDB0000224,0.029256,-0.051685,0.082465,-0.072923,0.059082
169,Arachidonic acid_HMDB0001043,-0.017747,-0.085023,0.103368,-0.028882,0.058755
184,Lathosterol_HMDB0001170,-0.067267,-0.064359,0.063621,0.038342,0.058397
...,...,...,...,...,...,...
63,Erythritol_HMDB0002994,-0.010464,0.014812,0.002040,-0.004377,0.007923
72,L-Cysteine_HMDB0000574,0.005397,-0.000337,-0.008746,0.011057,0.006384
12,2-Hydroxybutyric acid_HMDB0000008,0.006713,-0.005081,-0.002833,-0.009597,0.006056
156,myo-Inositol_HMDB0000211,-0.000193,0.011229,0.003227,-0.009399,0.006012


In [56]:
aha.sort_values(by='coef2',ascending=False)

,metabolite,coef0,coef1,coef2,coef3
169,Arachidonic acid_HMDB0001043,-0.017747,-0.085023,0.103368,-0.028882
162,Oleic acid_HMDB0000207,-0.005420,-0.064283,0.090704,-0.041268
161,Linoleic acid_HMDB0000673,-0.009084,-0.068950,0.087757,-0.030917
146,Palmitoleic acid_HMDB0003229,-0.013020,-0.052265,0.082790,-0.036143
114,O-Phosphoethanolamine_HMDB0000224,0.029256,-0.051685,0.082465,-0.072923
...,...,...,...,...,...
80,Phosphoenolpyruvic acid_HMDB0000263,0.013275,-0.040444,-0.064688,0.020393
115,3-Phosphoglyceric acid_HMDB0000807,0.015194,0.041438,-0.066676,-0.004442
52,Glutaric acid_HMDB0000661,0.030341,0.032728,-0.068898,-0.018572
148,Gluconic acid_HMDB0000625,0.005690,0.009438,-0.069886,0.022474


### Use metabolite value as X, storage condition as y, WHY?
- If use storage condition as X, metabolite as y, the model cannot tell whether the storage condition matter. The model can have good accuracy if the metabolite values are close to each other in each group.
- As I use metabolite value as X, storage condition as y, if the model can correctly predict storage condition with the metabolite value if and only if the metabolite value in different groups are very different.
- Since leave-one-out cross validation and some constraints are used to prevent overfitting, the model accuracy can be used to measure whether the value of a particular metabolite is affected by storage condition.


In [23]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

# loo = LeaveOneOut()
# loo.get_n_splits(X_centered)

# lsvc = LinearSVC(C = 0.05)
# lsvc.fit(X_centered[:,0].reshape(-1, 1), y)

target_names = ['2_8degC', 'RT', 'BHT', '-20degC']
metabolite_name = small_data.columns[1:]
test_metabolite = X_centered[:,0].reshape(-1, 1)
loo = LeaveOneOut()
loo.get_n_splits(test_metabolite)
acc_lst = []
macro_precision_lst = []
class_report = {}

print(loo)


for i in range(len(metabolite_name)):
    test_metabolite = X_centered[:,i].reshape(-1, 1)
    y_pred = []
    for train_index, test_index in loo.split(test_metabolite):

        X_train, X_test = test_metabolite[train_index], test_metabolite[test_index]
        y_train, y_test = y[train_index], y[test_index]

        lsvc = LinearSVC(C = 0.05,class_weight='balanced')
        lsvc.fit(X_train, y_train)

        pred = lsvc.predict(X_test)
        y_pred.append(pred[0])
    
    class_report[metabolite_name[i]] = y_pred
    acc_lst.append(accuracy_score(y, y_pred))
    macro_precision_lst.append(precision_score(y, y_pred, average='macro',zero_division=0))

LeaveOneOut()


In [24]:
metabolite_name

Index(['Dimethylglycine_HMDB0000092', 'Glyoxylic acid_HMDB0000119',
       'Pyruvic acid_HMDB0000243', 'L-Lactic acid_HMDB0000190',
       'Alpha-Hydroxyisobutyric acid_HMDB0000729', 'Glycolic acid_HMDB0000115',
       'Caproic acid_HMDB0000535', 'L-Alanine_HMDB0000161',
       'Alpha-ketoisovaleric acid_HMDB0000019', 'Glycine_HMDB0000123',
       ...
       'Sucrose_HMDB0000258', 'MG(18:0e/0:0/0:0)_HMDB0011143',
       'Trehalose_HMDB0000975', 'D-Maltose_HMDB0000163',
       'Lactitol_HMDB0040937', 'Xanthylic acid_HMDB0001554',
       'Adenosine monophosphate_HMDB0000045', 'Cholesterol_HMDB0000067',
       'Lathosterol_HMDB0001170', '7b-Hydroxycholesterol_ HMDB0006119'],
      dtype='object', length=186)

In [25]:
result = pd.DataFrame({'metabolite_name':metabolite_name,'accuracy':acc_lst,'macro_precision':macro_precision_lst})
result = result.sort_values(by='accuracy',ascending=False)
result.head()

,metabolite_name,accuracy,macro_precision
185,7b-Hydroxycholesterol_ HMDB0006119,0.466667,0.333333
138,D-Glucose_HMDB0000122.1,0.466667,0.283333
132,D-Glucose_HMDB0000122,0.466667,0.287500
91,Lyxose_HMDB0003402_1,0.400000,0.226190
153,D-Ribulose 5-phosphate_HMDB0000618,0.400000,0.238095


In [26]:
result.sort_values(by='macro_precision',ascending=False)

,metabolite_name,accuracy,macro_precision
185,7b-Hydroxycholesterol_ HMDB0006119,0.466667,0.333333
132,D-Glucose_HMDB0000122,0.466667,0.287500
138,D-Glucose_HMDB0000122.1,0.466667,0.283333
83,Triethanolamine_HMDB0032538,0.266667,0.250000
169,Arachidonic acid_HMDB0001043,0.200000,0.250000
...,...,...,...
160,Glutathione_HMDB0000125,0.000000,0.000000
177,MG(18:0e/0:0/0:0)_HMDB0011143,0.000000,0.000000
164,"alpha-Glucose 1,6-bisphosphate_HMDB0003514",0.000000,0.000000
165,Stearic acid_HMDB0000827,0.000000,0.000000


### Embedding results... from F-test and SVM
- F-test significant at 1%
- SVM accuracy > 0.3 (maybe not sufficient data)
- 0.3 as threshold: one dummy classifier always predict 1 can get accuracy of 0.26667


In [28]:
result_f = pd.read_excel('metabolite_f.xlsx').iloc[:,1:]
comb_f_svm = result.merge(result_f,how='left',left_on='metabolite_name',right_on='metabolite name')

In [32]:
comb_f_svm[(comb_f_svm['P_value']<=0.001)].sort_values(by='P_value',ascending=True)

,metabolite_name,accuracy,macro_precision,metabolite name,F_score,P_value
85,Arachidonic acid_HMDB0001043,0.200000,0.250000,Arachidonic acid_HMDB0001043,223.569290,3.894500e-10
11,2-Aminobenzoic acid_HMDB0001123,0.266667,0.142857,2-Aminobenzoic acid_HMDB0001123,190.517510,9.233940e-10
0,7b-Hydroxycholesterol_ HMDB0006119,0.466667,0.333333,7b-Hydroxycholesterol_ HMDB0006119,100.416068,2.830777e-08
184,Phosphoenolpyruvic acid_HMDB0000263,0.000000,0.000000,Phosphoenolpyruvic acid_HMDB0000263,97.183556,3.365516e-08
12,Triethanolamine_HMDB0032538,0.266667,0.250000,Triethanolamine_HMDB0032538,94.727047,3.852847e-08
48,L-Methionine_HMDB0000696,0.266667,0.142857,L-Methionine_HMDB0000696,83.481473,7.494935e-08
81,Oleic acid_HMDB0000207,0.200000,0.187500,Oleic acid_HMDB0000207,78.384008,1.042886e-07
25,4-Hydroxybenzoic acid_HMDB0000500,0.266667,0.111111,4-Hydroxybenzoic acid_HMDB0000500,74.054558,1.403604e-07
46,L-Phenylalanine_HMDB0000159,0.266667,0.142857,L-Phenylalanine_HMDB0000159,69.073935,2.017531e-07
38,Ornithine_HMDB0000214.1,0.266667,0.142857,Ornithine_HMDB0000214.1,68.651995,2.082858e-07


In [29]:
comb_f_svm[comb_f_svm['accuracy']>=0.3]

,metabolite_name,accuracy,macro_precision,metabolite name,F_score,P_value
0,7b-Hydroxycholesterol_ HMDB0006119,0.466667,0.333333,7b-Hydroxycholesterol_ HMDB0006119,100.416068,2.830777e-08
1,D-Glucose_HMDB0000122.1,0.466667,0.283333,D-Glucose_HMDB0000122.1,19.011598,1.164738e-04
2,D-Glucose_HMDB0000122,0.466667,0.287500,D-Glucose_HMDB0000122,20.356156,8.522593e-05
3,Lyxose_HMDB0003402_1,0.400000,0.226190,Lyxose_HMDB0003402_1,11.002692,1.222104e-03
4,D-Ribulose 5-phosphate_HMDB0000618,0.400000,0.238095,D-Ribulose 5-phosphate_HMDB0000618,20.216543,8.796409e-05
5,N-Acetylserine_HMDB0002931,0.400000,0.236111,N-Acetylserine_HMDB0002931,22.134784,5.783279e-05
6,L-Serine_HMDB0000187,0.333333,0.225000,L-Serine_HMDB0000187,21.755811,6.267218e-05
7,Succinic acid_HMDB0000254,0.333333,0.208333,Succinic acid_HMDB0000254,6.672117,7.884368e-03
8,N-Acetylglutamine_HMDB0006029,0.333333,0.194444,N-Acetylglutamine_HMDB0006029,19.245746,1.101653e-04


In [101]:
comb_f_svm[(comb_f_svm['P_value']<=0.001) & (comb_f_svm['accuracy']>=0.3)]

,metabolite_name,accuracy,metabolite name,F_score,P_value
0,7b-Hydroxycholesterol_ HMDB0006119,0.466667,7b-Hydroxycholesterol_ HMDB0006119,100.416068,2.830777e-08
1,D-Glucose_HMDB0000122.1,0.466667,D-Glucose_HMDB0000122.1,19.011598,1.164738e-04
2,D-Glucose_HMDB0000122,0.466667,D-Glucose_HMDB0000122,20.356156,8.522593e-05
4,D-Ribulose 5-phosphate_HMDB0000618,0.400000,D-Ribulose 5-phosphate_HMDB0000618,20.216543,8.796409e-05
5,N-Acetylserine_HMDB0002931,0.400000,N-Acetylserine_HMDB0002931,22.134784,5.783279e-05
6,L-Serine_HMDB0000187,0.333333,L-Serine_HMDB0000187,21.755811,6.267218e-05
8,N-Acetylglutamine_HMDB0006029,0.333333,N-Acetylglutamine_HMDB0006029,19.245746,1.101653e-04


In [116]:
met_lst = comb_f_svm[(comb_f_svm['P_value']<=0.001) & (comb_f_svm['accuracy']>=0.3)]['metabolite_name'].values

### $$ Recall = \frac{tp}{tp + fn}$$
### $$ Precision = \frac{tp}{tp + fp}$$
- fn:false negative, one sample is in class 1, but predicted class 2,3,4
- fp: false positive, one sample is in class 2,3,4, but predicted class 1
- the denominator of Recall is the number of samples really in class 1
- the denominator of Precision is the number of samples predicted class 1

In [117]:
for i in met_lst:
    print(i)
    print(classification_report(y,class_report[i],zero_division=0,target_names=target_names))
    print('\n')

7b-Hydroxycholesterol_ HMDB0006119
              precision    recall  f1-score   support

     2_8degC       0.33      1.00      0.50         4
          RT       0.00      0.00      0.00         4
         BHT       1.00      1.00      1.00         3
     -20degC       0.00      0.00      0.00         4

    accuracy                           0.47        15
   macro avg       0.33      0.50      0.38        15
weighted avg       0.29      0.47      0.33        15



D-Glucose_HMDB0000122.1
              precision    recall  f1-score   support

     2_8degC       0.80      1.00      0.89         4
          RT       0.33      0.75      0.46         4
         BHT       0.00      0.00      0.00         3
     -20degC       0.00      0.00      0.00         4

    accuracy                           0.47        15
   macro avg       0.28      0.44      0.34        15
weighted avg       0.30      0.47      0.36        15



D-Glucose_HMDB0000122
              precision    recall  f1-score  

In [14]:
import _thread
import time
 
# 为线程定义一个函数
def print_time( threadName, delay):
    count = 0
    while count < 5:
        time.sleep(delay)
        count += 1
        print( "%s: %s" % ( threadName, time.ctime(time.time()) ))

def time_alarm(start_time):
    now = time.time()
    if now-start_time >=10:
        return False
    
# 创建两个线程
try:
    _thread.start_new_thread( print_time, ("Thread-1", 2, ) )
    _thread.start_new_thread( print_aha, () )
except:
    print("Error: unable to start thread")
 


aha
Thread-1: Sun Oct  3 02:01:34 2021
Thread-1: Sun Oct  3 02:01:36 2021
Thread-1: Sun Oct  3 02:01:38 2021
Thread-1: Sun Oct  3 02:01:40 2021
Thread-1: Sun Oct  3 02:01:42 2021


In [22]:

class AdventureDone(Exception): pass

def time_alarm(start_time):
    now = time.time()
    if now-start_time >=10:
        print(now-start_time)
        _thread.exit()
        raise AdventureDone
        


start_time = time.time()
print(time.time()-start_time)
try:
    _thread.start_new_thread( time_alarm, (start_time,)) 
    while True:
        print('aha')
        time.sleep(5)
except AdventureDone:
    print('time')
    pass

0.0
aha
aha
aha
aha
aha


KeyboardInterrupt: 

In [24]:
start_time = time.time()
_thread.start_new_thread( time_alarm, (start_time,)) 

149440

In [46]:
def time_alarm(start_time):
    now = time.time()
    if now-start_time >=12:
        print(now-start_time)
        _thread.exit()
        raise AdventureDone
start_time = time.time()
time.sleep(5)
time.time() - start_time

5.0062575340271

In [3]:
import time
start_time = time.time()
exit_flag = False
while time.time()-start_time < 20:
    print('not yet')
    time.sleep(6)
    while True:
        if time.time()-start_time >= 15:
            print(time.time()-start_time)
            exit_flag = True
            break
    if exit_flag:
        print(time.time()-start_time)
        break

not yet
15.000209331512451
15.000209331512451
